<a href="https://colab.research.google.com/github/joshba06/Object_Detection/blob/main/PreparationAndTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Readme

**Step 1**: 

This script has to be run on both, Colab (training) and the local machine (object detection). Set the variable "system_id" to 1, if running on Colab or to 0 if running on local machine

# 1. Getting Started

## 1.1 Define parameters

In [1]:
# Choose 1, if running on colab and 0 if running on local machine
system_id = 1

# Define the objects that you would like to train the deep learning-model with below
labels = ['Car', 'Mug']

# Change model url and name if model changes
pre_trained_model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
pre_trained_model_name = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
custom_model_name = 'my_ssd_mobilenet_v2_fpnlite'

img_size = (640, 640)

## 1.2 Create file and folder structure

In [2]:
import os
import shutil

# Local machine
if system_id == 0:
    home_path = '/Users/niklas/Virtual_Environment/Version_4/Object_Detection'
    print('Running on local machine...')

# Google colab    
elif system_id == 1:
    
    print('Running on Google Colab...')
    
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Setup path to home directory 
    home_path = '/content/drive/MyDrive/Colab_Notebooks/Object_Detection'
    os.chdir(home_path)    

else:
    print('No operating system was defined...')

Running on Google Colab...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import setup
files, paths = setup.createFolderStructure(labels, home_path, system_id)

/content/drive/MyDrive/Colab_Notebooks/Object_Detection/0_User_Input already exists..
/content/drive/MyDrive/Colab_Notebooks/Object_Detection/0_User_Input/backgrounds already exists..
/content/drive/MyDrive/Colab_Notebooks/Object_Detection/0_User_Input/objects already exists..
/content/drive/MyDrive/Colab_Notebooks/Object_Detection/1_Preprocessing already exists..
/content/drive/MyDrive/Colab_Notebooks/Object_Detection/1_Preprocessing/images already exists..
Successfully created /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow from scratch. 
Successfully created /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/protoc from scratch. 
Successfully created /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/workspace from scratch. 
Successfully created /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/workspace/scripts from scratch. 
Successfully created /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Ten

In [4]:
# Create dict with labels
dict_labels = {}
num = 1
for label in labels:
    dict_labels[label]= num
    num += 1
print(dict_labels)

{'Car': 1, 'Mug': 2}


## 1.3 Installing Dependencies and importing modules

In [5]:
!pip uninstall opencv-python -y
!pip install opencv-python

Found existing installation: opencv-python 4.6.0.66
Uninstalling opencv-python-4.6.0.66:
  Successfully uninstalled opencv-python-4.6.0.66
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.9 MB)


In [6]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import cv2 as cv

import uuid

import time

import pathlib

import shutil

import math

import wget

# 3. Prepare Tensorflow Object Detection API

## 3.1 Install dependencies

In [8]:
# Local machine
if system_id == 0:
    !pip install tensorflow==2.5.0

# Google colab    
elif system_id == 1:
    !pip install tensorflow-gpu==2.5.0 

else:
    print('No operating system was defined...')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached absl_py-0.15.0-py3-none-any.whl (132 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.1.0
    Uninstalling absl-py-1.1.0:
      Successfully uninstalled absl-py-1.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-ei

In [9]:
# Download the model garden (model garden is an environment that is necessary to train new models from scratch or to continue training existing models)
# The model itself will be downloaded later

# Clone repository only if it does not exist already
os.chdir(paths['2_Tensorflow'])
if os.path.exists(paths['2_Tensorflow']+'/models/research') is False:
    print('Cloning model garden..')
    !git clone https://github.com/tensorflow/models.git
    
else:
    print('Model garden already exists')
    
os.chdir(paths['home'])

Cloning model garden..
Cloning into 'models'...
remote: Enumerating objects: 74953, done.
remote: Counting objects: 100% (441/441), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 74953 (delta 261), reused 386 (delta 230), pack-reused 74512
Receiving objects: 100% (74953/74953), 580.52 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (53161/53161), done.
Checking out files: 100% (3087/3087), done.


In [10]:
# Install protobuf
if os.path.exists(paths['2_Tensorflow']+'/protoc/protoc-21.1-osx-aarch_64.zip') is False:

    # Go to destination directory
    os.chdir(paths['protoc'])
    protoc_url = 'https://github.com/protocolbuffers/protobuf/releases/download/v21.1/protoc-21.1-osx-aarch_64.zip'
    wget.download(protoc_url)

    # Extract all content of downloaded file
    from zipfile import ZipFile

    with ZipFile('protoc-21.1-osx-aarch_64.zip', 'r') as zipObj:
        zipObj.extractall()

    os.environ['Path'] = paths['protoc']+'/bin'
    os.chdir(paths['research'])

    !protoc object_detection/protos/*.proto --python_out=.

else:
    print('Protobuf was already installed...')
    
os.chdir(paths['home'])

In [11]:
# Install pycocotools

# Clone repository only if it does not exist already
if os.path.exists(paths['research']+'/cocoapi') is False:
    print('Cloning cocoapi..')
    !git clone https://github.com/cocodataset/cocoapi.git
    
    # Moving cloned file to 'research' folder
    destination = paths['research']
    source = paths['home']+'/cocoapi'
    shutil.move(source, destination)
    
else:
    print('Cocoapi already exists')

os.chdir(paths['home'])

Cloning cocoapi..
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (576/576), done.


## 3.2 Install Tensorflow Object Detection API

In [12]:
# Check if API has already been installed
if os.path.exists(paths['research']+'/setup.py') is False:
    print('Installing setup.py...')
    
    # Move to 'research' directory
    os.chdir(paths['research'])

    # Copy setup.py to current working directory
    !cp object_detection/packages/tf2/setup.py .

    # Execute setup.py (this command installs all dependencies needed for tf2 odapi)
    !python -m pip install .

    print('Installation complete..')

else:
    print('Object Detection API has already been installed')

# Move back to home-directory
os.chdir(paths['home'])

Installing setup.py...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1694828 sha256=da021a888e013e66978f32f96291936e03da6ed46d0ebe507c57748054

## 3.3 Check if API was installed successfully

In [13]:
# Move to 'research' directory
os.chdir(paths['research'])
import object_detection

# Local machine
if system_id == 0:
    !python {paths['research']+'/object_detection/builders/model_builder_tf2_test.py'}
    

# Google colab    
elif system_id == 1:
    !pip install numpy --upgrade # This had to be added for execution on colab. Problem solved using stackoverflow

    !python {paths['research']+'/object_detection/builders/model_builder_tf2_test.py'}
else:
    print('No operating system was defined...')

# Move back to home directory
os.chdir(paths['home'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-07-06 15:35:07.860486: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0706 15:35:08.412369 140463042905984 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.49s
I0706 15:35:09.040414 140463042905984 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.49s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_

# 4. Prepare new training job

## 4.1 Install dependencies and import modules (only on Colab)

In [14]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
  
  ## Install missing modules for randomTrafficSign
  !pip install matplotlib
  !pip install lxml

  import numpy as np
  from xml.etree.ElementTree import ElementTree
  from xml.etree.ElementTree import Element
  import xml.etree.ElementTree as etree
  import xml.dom.minidom

  from lxml import etree
  os.chdir(paths['home'])

else:
    print('No operating system was defined...')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 4.2 Partition images for testing and training (only on Colab)

**Important**: Images of objects must be in the following format: "Mug_1.jpg", "Cat_3.jpg" and must be located in their respective folders

In [15]:
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
    !pip install -U albumentations --no-binary qudida,albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

    # imagePreprocessing.py must be in same directory as this main script
    %matplotlib inline
    path = paths['0_User_Input']+'/scripts'
    os.chdir(path)
    import imageProcessing as imgprep
    os.chdir(paths['home'])

    imgprep.clearPreprocessing(paths['images'])
    imgprep.clearTraining(paths['training']+'/images')

    numImgs = 150
    upperScale = 150
    lowerScale = 80

    # Modify and multiply images and store in 1_Preprocessing folder
    imgprep.main(dict_labels, paths['backgrounds'], paths['objects'], numImgs, upperScale, lowerScale, paths['images']+'/', paths['training']+'/')

    imgprep.clearPreprocessing(paths['images'])

Found 5 backgrounds
Found 7 objects for label: Car. Multiplying by factor 150
Multiplying image: Car_4.png
Multiplying image: Car_3.png
Multiplying image: Car_6.png
Multiplying image: Car_7.png
Multiplying image: Car_2.png
Multiplying image: Car_1.png
Multiplying image: Car_5.png
Multiplication completed. Currently available for label: Car, total: 1050, testing: 158, training: 892
Found 3 objects for label: Mug. Multiplying by factor 150
Multiplying image: Mug_3.png
Multiplying image: Mug_2.png
Multiplying image: Mug_1.png
Multiplication completed. Currently available for label: Mug, total: 450, testing: 68, training: 382


## 5.4 Create labelmap (only on Colab)

In [17]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

    # Convert label-dict to needed format
    labelmap = []
    for key in dict_labels:
      temp = {}
      temp['name'] = key
      temp['id'] = dict_labels[key]
      labelmap.append(temp)
    print(labelmap)

    files['labelmap'] = paths['annotations']+'/label_map.pbtxt'

    with open(files['labelmap'], 'w') as file:
      for label in labelmap:
          file.write('item { \n')
          file.write('\tname:\'{}\'\n'.format(label['name']))
          file.write('\tid:{}\n'.format(label['id']))
          file.write('}\n')
        
else:
    print('No operating system was defined...')

[{'name': 'Car', 'id': 1}, {'name': 'Mug', 'id': 2}]


In [18]:
# Install pandas
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 5.5 Create TFRecord (only on colab)

In [19]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  # Add labelmap and tfrecords to 'files' dictionary
  files['tf_train'] = paths['annotations']+'/train.record'
  files['tf_test'] = paths['annotations']+'/test.record'

  # Add line to download TF record file from nicknochnack

  # Copy generatetfrecord.py to scripts
  source = paths['0_User_Input']+'/scripts/generatetfrecord.py'
  shutil.copy(source, paths['scripts'])

  # Change directory to 'scripts'
  os.chdir(paths['workspace']+'/scripts')

  # Create / overwrite TFRecord files for training and testing

  # Create train data:
  !python generatetfrecord.py -x {paths['images_training']} -l {files['labelmap']} -o {files['tf_train']}

  # Create test data:
  !python generatetfrecord.py -x {paths['images_testing']} -l {files['labelmap']} -o {files['tf_test']}

  # Go back to home directory
  os.chdir(paths['home'])

 
else:
    print('No operating system was defined...')

Successfully created the TFRecord file: /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/workspace/training/annotations/train.record
Successfully created the TFRecord file: /content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/workspace/training/annotations/test.record


## 5.6 Download pre-trained model (only on colab)



In [20]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  # Update the settings for the image import and multiplication script depending on which size of image the model uses!

  # Check if the chosen model has already been downloaded
  if os.path.exists(paths['pre_trained_models']+'/'+str(pre_trained_model_name)) is False:

      # Go to destination directory
      os.chdir(paths['pre_trained_models'])
      wget.download(pre_trained_model_url)

      # Extract all content of downloaded file
      import tarfile

      file = tarfile.open('ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz')

      file.extractall(paths['pre_trained_models'])

      file.close()
                    
      # Delete downloaded tar.gz file to save storage space
      # Add code here
      #
      #
      
      # Create new folder for this model in training/models
      paths['active_model'] = paths['models']+'/'+custom_model_name
      os.makedirs(paths['active_model'])
      
      print('Model was successfully downloaded...')


  else:
    print(str(pre_trained_model_name)+' was already installed...')
    
    os.chdir(paths['home'])

else:
    print('No operating system was defined...')

Model was successfully downloaded...


## 5.7 Update the config file and pipeline for the new training job

In [21]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [22]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:

  ## Copy or replace pipeline in active model directory
  files['pipeline_downloaded'] = paths['pre_trained_models']+'/'+pre_trained_model_name+'/pipeline.config'
  paths['active_model'] = paths['models']+'/'+custom_model_name
  files['pipeline_active'] = paths['active_model']+'/pipeline.config'
  paths['downloaded_model'] = paths['pre_trained_models']+'/'+pre_trained_model_name

  # If pipeline already exists in active directory, replace it
  if os.path.exists(files['pipeline_active']) == True:
      os.remove(files['pipeline_active'])
      shutil.copy(files['pipeline_downloaded'], paths['active_model'])
      print('Pipeline replaced in active model directory...')

  # If pipeline does not yet exist in active directory, copy it from downloaded model
  else:
      files['pipeline_downloaded'] = paths['pre_trained_models']+'/'+pre_trained_model_name+'/pipeline.config' 
      shutil.copy(files['pipeline_downloaded'], paths['active_model'])
      print('Pipeline copied to active model directory...')

  ## Configure pipeline

  config = config_util.get_configs_from_pipeline_file(files['pipeline_active'])
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
  with tf.io.gfile.GFile(files['pipeline_active'], "r") as f:                                                                                                                                                                                                                     
      proto_str = f.read()                                                                                                                                                                                                                                          
      text_format.Merge(proto_str, pipeline_config)  


  pipeline_config.model.ssd.num_classes = len(labels) # Number of labels the model should be trained for
  pipeline_config.train_config.batch_size = 4 # This should be the number of training jobs that run parallel

  # Get checkpoint 0 from (original) downloaded model 
  files['checkpoint0'] = paths['downloaded_model']+'/checkpoint/ckpt-0'

  pipeline_config.train_config.fine_tune_checkpoint = files['checkpoint0']

  pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

  # Get labelmap
  pipeline_config.train_input_reader.label_map_path= files['labelmap']

  # Get TF-Record
  pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files['tf_train']]
  pipeline_config.eval_input_reader[0].label_map_path = files['labelmap']
  pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [files['tf_test']]

  config_text = text_format.MessageToString(pipeline_config)

  # Update active pipeline
  with tf.io.gfile.GFile(files['pipeline_active'], "wb") as f:                                                                                                                                                                                                                     
      f.write(config_text)   
      
  print('Pipeline successfully configured...')  

  # Copy model_main_tf2.py to workspace -> training   'TensorFlow/models/research/' file to 
  source = paths['research']+'/object_detection/model_main_tf2.py'
  destination = paths['training']
  shutil.copy(source, destination)

else:
    print('No operating system was defined...')


Pipeline copied to active model directory...
Pipeline successfully configured...


# 6. Start new training job (only on colab)

In [23]:
# Local machine
if system_id == 0:
    print('Skipping step')

# Google colab    
elif system_id == 1:
  
  files['training_script'] = paths['training']+'/model_main_tf2.py'
  model_dir = paths['active_model']
  
  command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(files['training_script'], model_dir, files['pipeline_active'])
  # first argument: Path to the model_main_tf2.py file
  # second argument: Path to the diretory in which the pipeline.config file is placed (not the path to the file itself)
  # third argument: Path to actual pipeline.config in active directory

  # This command is necessary to fix issue with training on colab
  # source: https://stackoverflow.com/questions/70998639/dnn-library-is-not-found-ssd-mobile-net-v2-in-colab#answer-72404540
  !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

  !{command}

else:
  print('No operating system was defined...')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
2022-07-06 15:38:04.940684: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0706 15:38:04.947924 139790858835840 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 10000
I0706 15:38:04.954953 139790858835840 config_util.py:552] Maybe overwriting train_steps: 10000
INFO:tensorflow:Maybe over

# 7 Evaluate training (both)

In [24]:
# Local machine
if system_id == 0:

  paths['train'] = paths['models']+'/my_ssd_mobilenet_v2_fpnlite/train'
  paths['eval'] = paths['models']+'/my_ssd_mobilenet_v2_fpnlite/eval'

  os.chdir(paths['train'])
  !tensorboard --logdir=.

# Google colab    
elif system_id == 1:

  command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(files['training_script'], model_dir, files['pipeline_active'], model_dir)
  !{command}

else:
  print('No operating system was defined...')

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/content/drive/MyDrive/Colab_Notebooks/Object_Detection/2_Tensorflow/workspace/training/model_main_tf2.py", line 89, in main
    wait_interval=300, timeout=FLAGS.eval_timeout)
  File "/usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py", line 1136, in eval_continuously
    checkpoint_dir, timeout=timeout, min_interval_secs=wait_interval):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/checkpoint_utils.py", line 195, in checkpoints_iterator
    checkpoint_dir, checkpoint_path, timeout=timeout)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/checkpoint_utils.py", line 143, in wait_for_new_checkpoint
    time.sleep(seconds_to_sleep)
KeyboardInterrupt

During handlin

# 8 Download model (Colab)

In [25]:
# Local machine
if system_id == 0:
  print('Skipping step...')

# Google colab    
elif system_id == 1:

    # Make zip file from directory
    import zipfile
    os.chdir(paths['models'])

    def zipdir(path, ziph):
      # ziph is zipfile handle
      for root, dirs, files in os.walk(path):
          for file in files:
              ziph.write(os.path.join(root, file), 
                        os.path.relpath(os.path.join(root, file), 
                                        os.path.join(path, '..')))

    with zipfile.ZipFile(custom_model_name+'.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
      zipdir(paths['active_model'], zipf)
        
    # Move zipfile to output folder 
    source = paths['active_model']+'/'+custom_model_name+'.zip'
    destination = paths['3_Output']
    shutil.move(source, destination)

    # Copy labelmap to export folder
    destination = paths['3_Output']
    source = files['labelmap']
    shutil.copy(source, destination)

else:
  print('No operating system was defined...')

FileNotFoundError: ignored